In [1]:
import time_series_transform as tst
import pandas as pd
import pandas_ta as ta
import numpy as np

In [2]:
port = tst.Stock_Transformer.from_stock_engine_period(
    symbols = ['GOOGL','NDAQ',"GOLD"],
    period = '5y',
    engine = 'yahoo'
)

In [3]:
strategy = ta.Strategy(
    name= 'mystrategy',
    ta=[
        {"kind": "ema", "length": 50},
        {"kind": "ema", "length": 7},
        {"kind": "ema", "length": 20},
        {"kind": "bbands", "length": 20},
        {"kind": "bbands", "length": 50},
        {"kind": "bbands", "length": 30},
        {"kind": "rsi","prefix":"rsi"},
        {"kind": "macd", "fast": 8, "slow": 21},
    ]
)
port = port.get_technial_indicator(strategy)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    1.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    2.3s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    3.5s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    3.5s finished


In [4]:
traget_list = ['Open','High','Low','Close','Volume','EMA_50','EMA_7',
     'EMA_20','BBL_20_2.0','BBM_20_2.0','BBU_20_2.0','BBL_50_2.0',
    'BBM_50_2.0',
    'BBU_50_2.0',
    'BBL_30_2.0',
    'BBM_30_2.0',
    'BBU_30_2.0',
    'rsi_RSI_14',
    'MACD_8_21_9',
    'MACDh_8_21_9',
    'MACDs_8_21_9']
port = port.make_lag_sequence(
    traget_list,
    windowSize = 60,
    lagNum = 1,
    suffix = '_lag_'
)
port = port.make_lead_sequence(traget_list,5,1,'_lead_')
port = port.dropna()
for t in traget_list:
    port = port.remove_feature(t)
df = port.to_pandas(True)
trans = tst.Time_Series_Transformer.from_pandas(df,'Date',None)
lag_cols = df.columns[df.columns.str.contains('_lag_')]
lead_cols = df.drop('Close_lead_5_GOOGL',axis =1).columns[df.drop('Close_lead_5_GOOGL',axis =1).columns.str.contains('_lead_')]
trans = trans.make_stack_sequence(list(lag_cols),'Lag_data')
for i in list(lead_cols):
    trans = trans.remove_feature(i)
for i in list(lag_cols):
    trans = trans.remove_feature(i)  

In [13]:
df = trans.to_pandas()
test = df.tail(100)
train = df.drop(test.index)

In [34]:
trainX = np.array(train['Lag_data'].tolist())
trainY = np.array(train['Close_lead_5_GOOGL'].tolist())
testX = np.array(test['Lag_data'].tolist())
testY = np.array(test['Close_lead_5_GOOGL'].tolist())

In [24]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow import keras
from time_series_transform import tfDataset_adopter as tda
from sklearn.metrics import classification_report,confusion_matrix

In [31]:
inputs = keras.Input(shape=(63,60),name = "Close_seq")
x = layers.LSTM(20)(inputs)
x = layers.Flatten()(inputs)
x = layers.Dense(10, activation="relu", name="dense_2")(x)
outputs = layers.Dense(5, name="change")(x)
model = keras.Model(inputs=inputs, outputs=outputs)
model.compile(
    optimizer = tf.keras.optimizers.RMSprop(learning_rate = 0.001),
    loss = tf.keras.losses.CategoricalCrossentropy()
)

In [33]:
model.fit(trainX,trainY,epochs = 1000)

Epoch 1/1000
33/33 [==============================] - 0s 1ms/step - loss: 22360.2773
Epoch 2/1000
33/33 [==============================] - 0s 1ms/step - loss: 38038.2930
Epoch 3/1000
33/33 [==============================] - 0s 1ms/step - loss: 28455.1504
Epoch 4/1000
33/33 [==============================] - 0s 1ms/step - loss: 28188.9785
Epoch 5/1000
33/33 [==============================] - 0s 937us/step - loss: 30202.8496
Epoch 6/1000
33/33 [==============================] - 0s 1ms/step - loss: 31624.9766
Epoch 7/1000
33/33 [==============================] - 0s 1ms/step - loss: 31384.6797
Epoch 8/1000
33/33 [==============================] - 0s 1ms/step - loss: 27365.3652
Epoch 9/1000
33/33 [==============================] - 0s 1ms/step - loss: 30737.1133
Epoch 10/1000
33/33 [==============================] - 0s 968us/step - loss: 27826.8164
Epoch 11/1000
33/33 [==============================] - 0s 1000us/step - loss: 34188.1914
Epoch 12/1000
33/33 [==============================] - 0

33/33 [==============================] - 0s 1ms/step - loss: 29345.9199
Epoch 97/1000
33/33 [==============================] - 0s 1ms/step - loss: 25085.4336
Epoch 98/1000
33/33 [==============================] - 0s 1ms/step - loss: 35333.7109
Epoch 99/1000
33/33 [==============================] - 0s 1ms/step - loss: 25341.2188
Epoch 100/1000
33/33 [==============================] - 0s 1ms/step - loss: 26874.2969
Epoch 101/1000
33/33 [==============================] - 0s 937us/step - loss: 31746.1191
Epoch 102/1000
33/33 [==============================] - 0s 1ms/step - loss: 30517.1289
Epoch 103/1000
33/33 [==============================] - 0s 1000us/step - loss: 26301.6094
Epoch 104/1000
33/33 [==============================] - 0s 1000us/step - loss: 29082.8730
Epoch 105/1000
33/33 [==============================] - 0s 1000us/step - loss: 33758.1875
Epoch 106/1000
33/33 [==============================] - 0s 1ms/step - loss: 25125.3770
Epoch 107/1000
33/33 [============================

33/33 [==============================] - 0s 1000us/step - loss: 38618.9922
Epoch 190/1000
33/33 [==============================] - 0s 1ms/step - loss: 38618.9922
Epoch 191/1000
33/33 [==============================] - 0s 969us/step - loss: 38618.9844
Epoch 192/1000
33/33 [==============================] - 0s 1000us/step - loss: 38618.9883
Epoch 193/1000
33/33 [==============================] - 0s 1ms/step - loss: 38618.9922
Epoch 194/1000
33/33 [==============================] - ETA: 0s - loss: 37121.851 - 0s 907us/step - loss: 38618.9922
Epoch 195/1000
33/33 [==============================] - 0s 938us/step - loss: 38618.9883
Epoch 196/1000
33/33 [==============================] - 0s 875us/step - loss: 38618.9844
Epoch 197/1000
33/33 [==============================] - 0s 937us/step - loss: 38618.9844
Epoch 198/1000
33/33 [==============================] - 0s 937us/step - loss: 38618.9922
Epoch 199/1000
33/33 [==============================] - 0s 906us/step - loss: 38618.9883
Epoch 200/

33/33 [==============================] - 0s 1ms/step - loss: 38618.9883
Epoch 282/1000
33/33 [==============================] - 0s 1ms/step - loss: 38618.9922
Epoch 283/1000
33/33 [==============================] - 0s 1ms/step - loss: 38618.9844
Epoch 284/1000
33/33 [==============================] - 0s 1ms/step - loss: 38618.9883
Epoch 285/1000
33/33 [==============================] - ETA: 0s - loss: 36095.019 - 0s 1ms/step - loss: 38618.9844
Epoch 286/1000
33/33 [==============================] - 0s 1ms/step - loss: 38618.9844
Epoch 287/1000
33/33 [==============================] - 0s 875us/step - loss: 38618.9922
Epoch 288/1000
33/33 [==============================] - 0s 875us/step - loss: 38618.9844
Epoch 289/1000
33/33 [==============================] - 0s 1ms/step - loss: 38618.9883
Epoch 290/1000
33/33 [==============================] - 0s 1ms/step - loss: 38618.9844
Epoch 291/1000
33/33 [==============================] - 0s 1ms/step - loss: 38618.9883
Epoch 292/1000
33/33 [====

33/33 [==============================] - 0s 1ms/step - loss: 38618.9883
Epoch 375/1000
33/33 [==============================] - 0s 1ms/step - loss: 38618.9883
Epoch 376/1000
33/33 [==============================] - 0s 1ms/step - loss: 38618.9883
Epoch 377/1000
33/33 [==============================] - 0s 965us/step - loss: 38618.9844
Epoch 378/1000
33/33 [==============================] - 0s 1ms/step - loss: 38618.9883
Epoch 379/1000
33/33 [==============================] - 0s 1ms/step - loss: 38618.9922
Epoch 380/1000
33/33 [==============================] - 0s 1ms/step - loss: 38618.9883
Epoch 381/1000
33/33 [==============================] - 0s 1ms/step - loss: 38618.9883
Epoch 382/1000
33/33 [==============================] - 0s 1ms/step - loss: 38618.9922
Epoch 383/1000
33/33 [==============================] - 0s 1ms/step - loss: 38618.9961
Epoch 384/1000
33/33 [==============================] - 0s 907us/step - loss: 38618.9883
Epoch 385/1000
33/33 [==============================] 

...

33/33 [==============================] - 0s 875us/step - loss: 38618.9883
Epoch 561/1000
33/33 [==============================] - 0s 1ms/step - loss: 38618.9883
Epoch 562/1000
33/33 [==============================] - 0s 1ms/step - loss: 38618.9883
Epoch 563/1000
33/33 [==============================] - 0s 1ms/step - loss: 38618.9883
Epoch 564/1000
33/33 [==============================] - 0s 1ms/step - loss: 38618.9883
Epoch 565/1000
33/33 [==============================] - 0s 1ms/step - loss: 38618.9883
Epoch 566/1000
33/33 [==============================] - 0s 1ms/step - loss: 38618.9844
Epoch 567/1000
33/33 [==============================] - 0s 1ms/step - loss: 38618.9844
Epoch 568/1000
33/33 [==============================] - 0s 976us/step - loss: 38618.9844
Epoch 569/1000
33/33 [==============================] - 0s 1ms/step - loss: 38618.9844
Epoch 570/1000
33/33 [==============================] - 0s 1ms/step - loss: 38618.9961
Epoch 571/1000
33/33 [==============================] 

33/33 [==============================] - 0s 969us/step - loss: 38618.9844
Epoch 654/1000
33/33 [==============================] - 0s 1ms/step - loss: 38618.9922
Epoch 655/1000
33/33 [==============================] - 0s 1ms/step - loss: 38618.9922
Epoch 656/1000
33/33 [==============================] - 0s 1000us/step - loss: 38618.9922
Epoch 657/1000
33/33 [==============================] - 0s 1ms/step - loss: 38618.9922
Epoch 658/1000
33/33 [==============================] - 0s 1ms/step - loss: 38618.9844
Epoch 659/1000
33/33 [==============================] - 0s 1ms/step - loss: 38618.9883
Epoch 660/1000
33/33 [==============================] - 0s 1ms/step - loss: 38618.9883
Epoch 661/1000
33/33 [==============================] - 0s 2ms/step - loss: 38618.9844
Epoch 662/1000
33/33 [==============================] - 0s 2ms/step - loss: 38618.9844
Epoch 663/1000
33/33 [==============================] - 0s 2ms/step - loss: 38618.9883
Epoch 664/1000
33/33 [==============================]

33/33 [==============================] - 0s 2ms/step - loss: 38618.9922
Epoch 747/1000
33/33 [==============================] - 0s 2ms/step - loss: 38618.9922
Epoch 748/1000
33/33 [==============================] - 0s 2ms/step - loss: 38618.9805
Epoch 749/1000
33/33 [==============================] - 0s 2ms/step - loss: 38618.9844
Epoch 750/1000
33/33 [==============================] - 0s 2ms/step - loss: 38618.9883
Epoch 751/1000
33/33 [==============================] - 0s 2ms/step - loss: 38618.9844
Epoch 752/1000
33/33 [==============================] - 0s 2ms/step - loss: 38618.9883
Epoch 753/1000
33/33 [==============================] - 0s 2ms/step - loss: 38618.9844
Epoch 754/1000
33/33 [==============================] - 0s 2ms/step - loss: 38618.9922
Epoch 755/1000
33/33 [==============================] - 0s 1ms/step - loss: 38618.9922
Epoch 756/1000
33/33 [==============================] - 0s 1ms/step - loss: 38618.9844
Epoch 757/1000
33/33 [==============================] - 0s

33/33 [==============================] - 0s 1ms/step - loss: 38618.9844
Epoch 841/1000
33/33 [==============================] - 0s 1ms/step - loss: 38618.9883
Epoch 842/1000
33/33 [==============================] - 0s 1ms/step - loss: 38618.9883
Epoch 843/1000
33/33 [==============================] - 0s 993us/step - loss: 38618.9844
Epoch 844/1000
33/33 [==============================] - 0s 937us/step - loss: 38618.9844
Epoch 845/1000
33/33 [==============================] - 0s 906us/step - loss: 38618.9844
Epoch 846/1000
33/33 [==============================] - 0s 1ms/step - loss: 38618.9844
Epoch 847/1000
33/33 [==============================] - 0s 2ms/step - loss: 38618.9922
Epoch 848/1000
33/33 [==============================] - 0s 2ms/step - loss: 38618.9883
Epoch 849/1000
33/33 [==============================] - 0s 1ms/step - loss: 38618.9883
Epoch 850/1000
33/33 [==============================] - 0s 1ms/step - loss: 38618.9922
Epoch 851/1000
33/33 [==============================

33/33 [==============================] - 0s 1ms/step - loss: 38618.9844
Epoch 935/1000
33/33 [==============================] - 0s 1ms/step - loss: 38618.9883
Epoch 936/1000
33/33 [==============================] - 0s 1ms/step - loss: 38618.9883
Epoch 937/1000
33/33 [==============================] - 0s 2ms/step - loss: 38618.9844
Epoch 938/1000
33/33 [==============================] - 0s 1ms/step - loss: 38618.9883
Epoch 939/1000
33/33 [==============================] - 0s 1ms/step - loss: 38618.9883
Epoch 940/1000
33/33 [==============================] - 0s 1ms/step - loss: 38618.9922
Epoch 941/1000
33/33 [==============================] - 0s 1ms/step - loss: 38618.9883
Epoch 942/1000
33/33 [==============================] - 0s 1000us/step - loss: 38618.9805
Epoch 943/1000
33/33 [==============================] - 0s 1000us/step - loss: 38618.9883
Epoch 944/1000
33/33 [==============================] - 0s 1ms/step - loss: 38618.9844
Epoch 945/1000
33/33 [==============================

In [35]:
model.predict(testX)

array([[ 18087646. , -27462952. ,  15654511. ,  -4400430. ,  -4012286.2],
       [ 17905512. , -26853360. ,  15424557. ,  -5094156. ,  -4663639.5],
       [ 18076512. , -27234582. ,  15598957. ,  -4866321.5,  -4449031. ],
       [ 18241008. , -27682938. ,  15784448. ,  -4466256.5,  -4073032.8],
       [ 18082484. , -27755094. ,  15715181. ,  -3734428.5,  -3388082.2],
       [ 17856260. , -25847276. ,  15179704. ,  -7145919.5,  -6587040.5],
       [ 18835450. , -26587222. ,  15865016. ,  -9038989. ,  -8355322. ],
       [ 19648260. , -29006088. ,  16825748. ,  -6611351.5,  -6074880. ],
       [ 18686354. , -28577256. ,  16217254. ,  -4091216.8,  -3718750.2],
       [ 18761738. , -28763830. ,  16298157. ,  -3949745.2,  -3585683.5],
       [ 18085280. , -26897900. ,  15530546. ,  -5644035. ,  -5177920. ],
       [ 19250634. , -28497266. ,  16502219. ,  -6304310.5,  -5789560.5],
       [ 18723922. , -27991686. ,  16110231. ,  -5524364. ,  -5061791.5],
       [ 18438828. , -28472830. ,  160